In [4]:
import glob
import datetime
import os
import pandas as pd

def get_traveler_name(text_pages):
    all_names = []
    names = []
    print(len(text_pages))
    for i in range(len(text_pages)):
        # print('hello')
        page = text_pages[i].splitlines()
        page = [line for line in page if not line.isspace() and len(line) > 0]
        for j in range(len(page)):
            if 'cab' in page[j].lower() and 'ms.' not in page[j].lower() and 'number' not in page[j].lower() and 'number' not in page[j+1].lower() and 'beach' not in page[j].lower() and 'airport' not in page[j+1].lower() and 'booking' not in page[j+1].lower() and 'booking' not in page[j+2].lower() and 'toll' not in page[j+1].lower() and 'one way' not in page[j+1].lower() and 'one way' not in page[j].lower():
                if 'mr.' not in page[j].lower():
                    for k in range(j+1, len(page)):
                        if 'guide' in page[k].lower() or 'service' in page[k+1].lower():
                            break
                        if 'sector' in page[k].lower() or 'travel' in page[k].lower() or 'destination' in page[k].lower():
                            # print('names---> ', names)
                            all_names.append(names)
                            names = []
                            break
                        else:
                            # if 'mr' in page[k].lower() or 'ms' in page[k].lower():
                            if '-' not in page[k] and 'days' not in page[k].lower():
                                names.append(page[k])
                else:
                    name = ''
                    for k in range(j, len(page)):
                        if 'sector' in page[k].lower():
                            # print('names---> ', names)
                            all_names.append(names)
                            names = []
                            break
                        lower_string = page[k].lower()
                        print('lower_string---> ', lower_string)
                        # print('lower string words---> ', lower_string.split())
                        ind = lower_string.split().index('mr.')
                        # print('ind---> ', ind)
                        words = page[k].split()[ind:]
                        # print(words)
                        for word in words:
                            if not word.strip('.').isalpha():
                                # print('name---> ', name)
                                break
                            name += word + ' '
                        # # print(name)
                        if name:
                            names.append(name.strip())
                            name = ''
                        # else:
                        #     names.append(page[k])
            
            if 'balance due' in page[j].lower() or 'terms & conditions' in page[j].lower():
                all_names = clean_traveler_name(all_names)
                return all_names

            
    # [['Mr. Siddharth Goyal', 'Mr. Ajay Sharma'], ['Preferred Seat', 'Selection Charges Inc.', 'Sc'], ['Mr. Siddharth Goyal', 'Mr. Ajay Sharma']]
            
def clean_traveler_name(all_names):

    debarred = ['preferred', 'selection', 'air', 'chares', 'sc', 'booking', 'jaipur', 'cab', 'delhi', 'package', 'cgst']
    
    for x in all_names.copy():
        for y in x.copy():
            for d in debarred:
                if d in y.lower():
                    x.remove(y)
                    break

    all_names = [sublist for sublist in all_names if sublist]

    for i in range(len(all_names.copy())):
        for j in range(len(all_names[i].copy())):
            if 'mr.' not in all_names[i][j].lower() and 'ms.' not in all_names[i][j].lower() and not all_names[i][j].isnumeric():
                all_names[i][j-1] = all_names[i][j-1]+' '+all_names[i][j]
                all_names[i].remove(all_names[i][j])
                
    # [['Mr. Devendra Natani', 'Mr. Abhijeet Gupta'], ['Mr. Rohit Mehta'], ['Mr. Rohit Mehta'], ['Mr. Devendra Natani'], ['Mr. Satish & Mr. Vipin'], ['Mr. Vishal Soni', 'Ms. Shilpi Jain'], ['Mr. Vikas Mittal'], ['Mr. Vikas Mittal']]

    for i in range(len(all_names)):
        all_names[i] = ' & '.join(all_names[i])

    # print('all names---> ', all_names)

    return all_names

def get_start_date(text_pages):
    start_date = ''
    start_dates = []
    for i in range(len(text_pages)):
        page = text_pages[i].splitlines()
        page = [line for line in page if not line.isspace() and len(line) > 0]
        for j in range(len(page)):
            if 'cab' in page[j].lower() and 'ms.' not in page[j].lower() and 'cgst' not in page[j+1].lower() and 'beach' not in page[j].lower() and 'airport' not in page[j+1].lower() and 'booking' not in page[j+1].lower() and 'booking' not in page[j+2].lower() and 'toll' not in page[j+1].lower() and 'one way' not in page[j+1].lower() and 'one way' not in page[j].lower() and 'travel date' not in page[j+1].lower() and 'sector' not in page[j+1].lower() and 'local' not in page[j].lower() and 'local' not in page[j+2].lower():
                start_date = ''
                flag = 0
                for k in range(j+1, len(page)):
                    if ('travel date' in page[k].lower() and not 'old travel date' in page[k].lower()) or ('date' in page[k].lower() and not 'old travel date' in page[k].lower()):
                        for t in range(k, len(page)):
                            if 'cab' in page[t].lower() or 'return' in page[t].lower() or 'sector' in page[t].lower() or 'purpose' in page[t].lower() or 'one way' in page[t].lower():
                                start_dates.append(start_date.strip())
                                start_date = ''
                                flag = 1
                                break
                            if 'date' in page[t].lower():
                                start_date += page[t].split(':')[-1]+' '
                                # print('start date---> ', start_date)
                            else:
                                start_date += page[t] + ' '
                    if flag:
                        break
            
            if 'balance due' in page[j].lower() or 'terms & conditions' in page[j].lower():
                start_dates = clean_date(start_dates)
                # print('start dates---> ', start_dates)
                return start_dates
            
def get_end_date(dates):
    end_date = ''
    end_dates = []
    flag = 0
    for i in range(len(text_pages)):
        page = text_pages[i].splitlines()
        page = [line for line in page if not line.isspace() and len(line) > 0]
        for j in range(len(page)):
            if 'cab' in page[j].lower() and 'ms.' not in page[j].lower() and 'beach' not in page[j].lower() and 'airport' not in page[j+1].lower() and 'booking' not in page[j+1].lower() and 'booking' not in page[j+2].lower() and 'toll' not in page[j+1].lower() and 'one way' not in page[j+1].lower():
                end_date = ''
                flag = 0
                for k in range(j+1, len(page)):
                    if 'cab' in page[k].lower():
                        break
                    if 'return' in page[k].lower():
                        for t in range(k, len(page)):
                            if 'cab' in page[t].lower() or 'for' in page[t].lower():
                                end_dates.append(end_date.strip())
                                end_date = ''
                                flag = 1
                                break
                            if 'return' in page[t].lower():
                                end_date += page[t].split(':')[-1]+' '
                                # print('start date---> ', start_date)
                            else:
                                end_date += page[t] + ' '
                    if flag:
                        break

            
            if 'balance due' in page[j].lower() or 'terms & conditions' in page[j].lower():
                end_dates = clean_date(end_dates)
                return end_dates
            
def clean_date(dates):
    extra_ch = ['*', '’', ' ', '\'']
    for i in range(len(dates)):
        for ec in extra_ch:
            if ec in dates[i]:
                dates[i] = dates[i].replace(ec, '')

    for i in range(len(dates)):
        if '-' not in dates[i]:
            dates[i] = dates[i][:2]+' '+dates[i][2:5]+' '+dates[i][5:]

    formats = ['%d %b %Y', '%d %b %y', '%d-%m-%Y', '%d-%m-%y']
    for i in range(len(dates)):
        for format in formats:
            try:
                dates[i] = datetime.datetime.strptime(dates[i], format).strftime('%Y-%m-%d')
            except:
                pass

    return dates
            
def get_final_start_dates(start_dates, traveler_names):
    final_start_dates = []
    for i in range(len(traveler_names)):
        if start_dates:
            final_start_dates.extend([start_dates[i]]*len(traveler_names[i]))
        else:
            final_start_dates.extend(['None']*len(traveler_names[i]))
    return final_start_dates

def get_total_cost(text_pages):
    costs = []
    chs = [',', '.']
    for i in range(len(text_pages)):
        page = text_pages[i].splitlines()
        page = [line for line in page if not line.isspace() and len(line) > 0]
        for j in range(len(page)):
            if 'cab' in page[j].lower() and 'ms.' not in page[j].lower() and 'number' not in page[j].lower() and 'number' not in page[j+1].lower() and 'cgst' not in page[j+1].lower() and 'beach' not in page[j].lower() and 'airport' not in page[j+1].lower() and 'booking' not in page[j+1].lower() and 'booking' not in page[j+2].lower() and 'toll' not in page[j+1].lower() and 'one way' not in page[j+1].lower() and 'one way' not in page[j].lower() and 'travel date' not in page[j+1].lower() and 'sector' not in page[j+1].lower() and 'local' not in page[j].lower():
                words = page[j].split()
                num = words[-1]
                for ch in chs:
                    num = num.replace(ch, '')
                    # print('num---> ', num)
                if num.isnumeric():
                    costs.append(words[-1])
    return costs
                        
all_traveler_names = []
all_start_dates = []
all_end_dates = []
all_filenames = []
all_costs = []

for file in glob.glob('C:\\air_ticket\\appcino_all_docs\\ensemble_files3\\*'):
    # if '474' not in file:
    print(file.split('\\')[-1])
    filename = os.path.split(file)[-1]
    data = pd.read_excel('World_DB.xlsx')
    
    with open(file) as fp:
        text = fp.read()
        text_pages = text.split('---')[:-1]

        traveler_names = get_traveler_name(text_pages)
        print('traveler names---> ', traveler_names)
        # new_traveler_names = [item for sublist in traveler_names for item in sublist]

        # start_dates = get_start_date(text_pages)
        # start_dates.extend(['None']*(len(traveler_names)-len(start_dates)))
        # final_start_dates = get_final_start_dates(start_dates, traveler_names)

        # end_dates = get_end_date(text_pages)
        # end_dates.extend(['None']*(len(start_dates)-len(end_dates)))
        # final_end_dates = get_final_start_dates(end_dates, traveler_names)

        # costs = get_total_cost(text_pages)

        # traveler_names.extend(['None']*(len(start_dates)-len(traveler_names)))

        # print('names---> ', len(new_traveler_names))
        # print('final start dates---> ', len(final_start_dates))
        # print('final end dates---> ', len(final_end_dates))

        # print('names---> ', traveler_names)
        # print('final start dates---> ', start_dates)
        # print('final end dates---> ', end_dates)
        # print('cost---> ', costs)

        print()

        # tnames_total = []
        # for i in range(len(new_traveler_names)):
        #     fn = filename.split('.txt')[0]
        #     all_filenames.extend([fn]*l)
        #     all_traveler_names.extend([new_traveler_names[i]]*l)
        #     tnames_total = [new_traveler_names[i]]*l
        #     # invoice_sheet.extend(invoice_final*len(tnames_total))
        #     all_start_dates.extend([final_start_dates[i]]*l)
        #     all_end_dates.extend([final_end_dates[i]]*l)
        #     invoice_date_sheet.extend([invoice_date_final]*l)

        # all_filenames.extend([filename.split('.txt')[0]]*len(traveler_names))
        # all_traveler_names.extend(traveler_names)
        # all_start_dates.extend(start_dates)
        # all_end_dates.extend(end_dates)
        # all_costs.extend(costs)


        # if len(traveler_names) != len(costs):
        #     print(traveler_names)
        #     print(len(traveler_names))
        #     print(start_dates)
        #     print(len(start_dates))
        #     print(end_dates)
        #     print(len(end_dates))
        #     print(costs)
        #     print(len(costs))

# print(len(all_filenames))
# print(len(all_traveler_names))             
# print(len(all_start_dates))
# print(len(all_end_dates))
# print(len(all_costs))

#342-2023_Appcino_Technologies.txt
4
lower_string--->  4 20/12/2022 cab mr. miguel angel paraddo 1,380.00 1,380.00
lower_string--->  destination : jaipur


ValueError: 'mr.' is not in list